# 🤖 Model Inference Notebook

This notebook demonstrates how to use your trained loan evaluation model for inference.

In [1]:
# Setup and imports
import sys
import os
import json

# Add project root to path
project_root = os.path.dirname(os.getcwd())
sys.path.insert(0, project_root)

from src.inference import LoanEvaluatorInference
from config import OUTPUT_CONFIG, MODEL_CONFIG

📁 Directory exists: /Users/shamsulhaq/Desktop/SNH_AI_CODING/outputs
📁 Directory exists: /Users/shamsulhaq/Desktop/SNH_AI_CODING/checkpoints
📁 Directory exists: /Users/shamsulhaq/Desktop/SNH_AI_CODING/logs
📁 Directory exists: /Users/shamsulhaq/Desktop/SNH_AI_CODING/models/Llama-3.2-1B-Instruct
📁 Directory exists: /Users/shamsulhaq/Desktop/SNH_AI_CODING/utils
📁 Directory exists: /Users/shamsulhaq/Desktop/SNH_AI_CODING/scripts
📁 Directory exists: /Users/shamsulhaq/Desktop/SNH_AI_CODING/experiments
📁 Directory exists: /Users/shamsulhaq/Desktop/SNH_AI_CODING/plots


In [2]:
# Initialize the inference model
print("🤖 Loading model for inference...")

# Try to load your trained model first
trained_model_path = os.path.join(OUTPUT_CONFIG["output_dir"], "final_model")

if os.path.exists(trained_model_path):
    print(f"✅ Found trained model at: {trained_model_path}")
    inferencer = LoanEvaluatorInference(trained_model_path)
else:
    print(f"⚠️  No trained model found. Using base model: {MODEL_CONFIG['model_id']}")
    inferencer = LoanEvaluatorInference()

print("✅ Model loaded and ready!")

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


🤖 Loading model for inference...
✅ Found trained model at: /Users/shamsulhaq/Desktop/SNH_AI_CODING/outputs/final_model
🤖 Loading model from: /Users/shamsulhaq/Desktop/SNH_AI_CODING/outputs/final_model
📱 Using device: mps


ImportError: 
 requires the protobuf library but it was not found in your environment. Check out the instructions on the
installation page of its repo: https://github.com/protocolbuffers/protobuf/tree/master/python#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.


## 🧪 Test with Sample Applications

In [ ]:
# Example 1: Strong Application (should be APPROVED)
strong_application = {
    "age": 35,
    "credit_score": 780,
    "annual_income_usd": 85000,
    "debt_to_income_ratio_percent": 15.0,
    "employment_status": "employed_full_time",
    "current_employment_duration_months": 36,
    "residency_status": "US_Citizen",
    "has_bankruptcy_recent": False,
    "requested_amount_usd": 30000,
    "has_verifiable_bank_account": True
}

print("💪 Testing Strong Application:")
print(json.dumps(strong_application, indent=2))

result = inferencer.evaluate_loan_application(strong_application)
print("\n🎯 Result:")
print(json.dumps(result["prediction"], indent=2))

In [ ]:
# Example 2: Weak Application (should be REJECTED)
weak_application = {
    "age": 22,
    "credit_score": 580,  # Below 670
    "annual_income_usd": 25000,  # Below 30k
    "debt_to_income_ratio_percent": 45.0,  # Above 40%
    "employment_status": "employed_part_time",
    "current_employment_duration_months": 3,  # Below 6 months
    "residency_status": "US_Citizen",
    "has_bankruptcy_recent": False,
    "requested_amount_usd": 40000,
    "has_verifiable_bank_account": True
}

print("⚠️ Testing Weak Application:")
print(json.dumps(weak_application, indent=2))

result = inferencer.evaluate_loan_application(weak_application)
print("\n🎯 Result:")
print(json.dumps(result["prediction"], indent=2))

In [ ]:
# Example 3: Borderline Application (should be FLAG_REVIEW)
borderline_application = {
    "age": 28,
    "credit_score": 690,  # Just above threshold
    "annual_income_usd": 55000,
    "debt_to_income_ratio_percent": 35.0,
    "employment_status": "self_employed",
    "current_employment_duration_months": 4,  # Below 6 months - FLAG
    "residency_status": "Permanent_Resident",
    "has_bankruptcy_recent": False,
    "requested_amount_usd": 35000,  # Above 50% of income - FLAG
    "has_verifiable_bank_account": True
}

print("🤔 Testing Borderline Application:")
print(json.dumps(borderline_application, indent=2))

result = inferencer.evaluate_loan_application(borderline_application)
print("\n🎯 Result:")
print(json.dumps(result["prediction"], indent=2))

## 🎮 Interactive Testing

Create your own loan application and test it!

In [ ]:
# Create your own application
custom_application = {
    "age": 45,  # Change these values
    "credit_score": 720,
    "annual_income_usd": 70000,
    "debt_to_income_ratio_percent": 30.0,
    "employment_status": "employed_full_time",  # employed_full_time, employed_part_time, self_employed, retired
    "current_employment_duration_months": 24,
    "residency_status": "US_Citizen",  # US_Citizen, Permanent_Resident
    "has_bankruptcy_recent": False,  # True or False
    "requested_amount_usd": 25000,
    "has_verifiable_bank_account": True  # True or False
}

print("🎮 Your Custom Application:")
print(json.dumps(custom_application, indent=2))

result = inferencer.evaluate_loan_application(custom_application)
print("\n🎯 Result:")
print(json.dumps(result["prediction"], indent=2))

## 📊 Batch Testing

Test multiple applications at once

In [ ]:
# Load and test multiple applications from test data
from config import get_data_paths

data_paths = get_data_paths()
test_file = data_paths["test"]

if os.path.exists(test_file):
    print(f"📁 Loading test data from: {test_file}")
    
    test_results = []
    
    with open(test_file, 'r') as f:
        for i, line in enumerate(f):
            if i >= 5:  # Test first 5 samples
                break
                
            sample = json.loads(line.strip())
            
            # Extract user message (loan application)
            for message in sample["messages"]:
                if message["role"] == "user":
                    try:
                        app_data = json.loads(message["content"])
                        result = inferencer.evaluate_loan_application(app_data)
                        
                        # Get ground truth
                        ground_truth = None
                        for msg in sample["messages"]:
                            if msg["role"] == "assistant":
                                ground_truth = msg["content"]
                                break
                        
                        test_results.append({
                            "sample": i + 1,
                            "prediction": result["prediction"],
                            "ground_truth": ground_truth
                        })
                        
                        print(f"\n📋 Sample {i + 1}:")
                        print(f"Prediction: {result['prediction']}")
                        print(f"Ground Truth: {ground_truth}")
                        
                    except Exception as e:
                        print(f"❌ Error processing sample {i + 1}: {e}")
                    break
    
    print(f"\n✅ Processed {len(test_results)} test samples")
else:
    print(f"❌ Test file not found: {test_file}")